In [1]:
import requests
from bs4 import BeautifulSoup as soup
import os, time
import json,re

In [2]:
filmlk_url = 'https://www.films.lk/'

In [3]:
filename = "artists-corpus/artists_links.csv"

def scape_main_pages():
    with open(filename,'w') as f:
        for i in range(1,30):
            main_url = 'https://www.films.lk/artists.php?page={}'.format(i)
            print("scrapping : " + main_url)
            response = requests.get(
                url = main_url,
            )

            page_soup = soup(response.content, 'html.parser')
            artists = page_soup.findAll("div",{"class":"column2"})


            for artist in artists:
                name = artist.span.text
                url = artist.a["href"]
                f.write(url + "\n")

            time.sleep(10)

In [7]:
scape_main_pages()

scrapping : https://www.films.lk/artists.php?page=1
scrapping : https://www.films.lk/artists.php?page=2
scrapping : https://www.films.lk/artists.php?page=3
scrapping : https://www.films.lk/artists.php?page=4
scrapping : https://www.films.lk/artists.php?page=5
scrapping : https://www.films.lk/artists.php?page=6
scrapping : https://www.films.lk/artists.php?page=7
scrapping : https://www.films.lk/artists.php?page=8
scrapping : https://www.films.lk/artists.php?page=9
scrapping : https://www.films.lk/artists.php?page=10
scrapping : https://www.films.lk/artists.php?page=11
scrapping : https://www.films.lk/artists.php?page=12
scrapping : https://www.films.lk/artists.php?page=13
scrapping : https://www.films.lk/artists.php?page=14
scrapping : https://www.films.lk/artists.php?page=15
scrapping : https://www.films.lk/artists.php?page=16
scrapping : https://www.films.lk/artists.php?page=17
scrapping : https://www.films.lk/artists.php?page=18
scrapping : https://www.films.lk/artists.php?page=19
sc

In [4]:
def get_artist_list(start, end): 
    with open(filename, "r") as f:
        artists_urls = f.readlines()
    
    artist_list = []
    
    for i in range(start, end):
        response = requests.get(
            url = filmlk_url + artists_urls[i].rstrip('\n'),
        )
  
        artist_data = get_artist_meta_data(response.content) 
    
        # biography
        artist_bio_data = get_artist_bio_data(response.content) 
        artist_data['biography_en'] = artist_bio_data
        
        # awards
        artist_award_data = get_artist_award_data(response.content) 
        artist_data['national_awards_en'] = artist_award_data
        
        # filmography
        artist_film_data = get_artist_film_data(response.content)  
        
        if (len(artist_film_data) < 1):
            continue
        
        artist_data['filmography_en'] = artist_film_data
        
        with open ('artists-corpus/artists_data.json','r') as f:
            data = json.load(f)
        
        data.append(artist_data)
        
        with open('artists-corpus/artists_data.json', "w") as f:
            json.dump(data, f)
                  
        artist_list.append(artist_data)
          
        time.sleep(5)
        
    return artist_list            

In [5]:
def get_artist_meta_data(content):
    
    page_soup = soup(content, 'html.parser')
    artist = page_soup.findAll("div",{"class":"column2"})
    artist_summary = artist[1].findAll("li",{"class":None})
        
    artist_meta_data = {}
    artist_meta_data["known_as_en"] = ""
    artist_meta_data["known_as_si"] = ""
    artist_meta_data["real_name_en"] = ""
    artist_meta_data["birth_en"] = ""
    artist_meta_data["death_en"] = ""
    
    for field in artist_summary:
        if field.b != None:

            if "Known As" in field.b.get_text():
                known_as_en = field.text.split(":")[1].strip()
                print("scrapping : " + known_as_en)
                artist_meta_data["known_as_en"] = known_as_en

            if "Sinhala" in field.b.get_text(): 
                known_as_si = field.text.split(":")[1].strip()
                artist_meta_data["known_as_si"] = known_as_si

            if "Real Name" in field.b.get_text(): 
                real_name = field.text.split(":")[1].strip()
                artist_meta_data["real_name_en"] = real_name

            if "Birthday" in field.b.get_text(): 
                birth = field.text.split(":")[1].strip() 
                artist_meta_data["birth_en"] = birth

            if "Died" in field.b.get_text(): 
                death = field.text.split(":")[1].strip()
                artist_meta_data["death_en"] = death
    
    return artist_meta_data
    

In [6]:
def get_artist_film_data(content):
    
    page_soup = soup(content, 'html.parser')
    columns = page_soup.findAll("div",{"class":"column"})
    artist_film_data = []
    
    for column in columns:
        field = column.find("span",{"class":"eventTitle"})   

        if (field != None and field.get_text() == 'FILMOGRAPHY'): 

            titles = (column.findAll("span",{"class":"eventTitle"}))

            titles = list(map(lambda title: title.get_text(), titles))

            if 'AS CAST MEMBER' not in titles:
                continue

            td_rows = column.table.findAll("td",{"class":"row1"})

            for row in td_rows:

                if (row.span != None and row.a != None):
                    filmography = {}

                    role_name = row.span.text
                    film_name = row.a.text

                    filmography["film_name_en"] = film_name
                    filmography["role_name_en"] = role_name

                    artist_film_data.append(filmography) 
                    
    return artist_film_data

In [7]:
def get_artist_bio_data(content):
    page_soup = soup(content, 'html.parser')
    columns = page_soup.findAll("div",{"class":"column"})
    artist_bio_data = ""
    
    for column in columns:
        field = column.find("span",{"class":"eventTitle"})   

        if (field != None and field.get_text() == 'BIOGRAPHY'): 

            for p in column.findAll('p',{"class":None}):
                artist_bio_data += (p.text)
                
    return artist_bio_data                

In [8]:
def get_artist_award_data(content):
    page_soup = soup(content, 'html.parser')
    columns = page_soup.findAll("div",{"class":"column"})
    rows = page_soup.findAll("div",{"class":"row"})
    
    artist_award_data = []
    
    for column in columns:
        field = column.find("span",{"class":"eventTitle"})   

        if (field != None and field.get_text() == 'NATIONAL AWARDS'):

            awards = column.findAll("div",{"class":"column22"})

            for award in awards:  
                award_x = {}
                award_x['award_name_en'] = award.p.text.split(' ',1)[1]
                award_x['award_ceremony_name_en'] = award.findAll("a")[0].text
                award_x['film_name_en'] = award.findAll("a")[1].text

                artist_award_data.append(award_x) 

    for row in rows:
        field = row.find("span",{"class":"eventTitle"})   

        if (field != None and field.get_text() == 'NATIONAL AWARDS'):

            awards = row.findAll("div",{"class":"column22"})

            for award in awards:  
                award_x = {}
                award_x['award_name_en'] = award.p.text.split(' ',1)[1]
                award_x['award_ceremony_name_en'] = award.findAll("a")[0].text
                award_x['film_name_en'] = award.findAll("a")[1].text

                artist_award_data.append(award_x) 
                
    return artist_award_data   

In [116]:
artist_list = get_artist_list(0,100)

scrapping : Rukmani Devi
scrapping : D.R. Nanayakkara
scrapping : B.A.W. Jayamanne
scrapping : Eddy Jayamanna
scrapping : Clarice De Silva
scrapping : Sandya Kumari
scrapping : Prem Jayanth
scrapping : Ravindra Rupasena
scrapping : Gamini Fonseka
scrapping : Vijaya Kumaranatunga
scrapping : Joe Abeywickrama
scrapping : Dr. Lester James Peries
scrapping : Malani Fonseka
scrapping : Geetha Kumarasinghe
scrapping : Swarna Mallawarachchi
scrapping : Anoja Weerasinghe
scrapping : Freddie Silva
scrapping : Sanath Gunathilaka
scrapping : Sabeetha Perera
scrapping : Jeewan Kumaranatunga
scrapping : Pooja Umashankar
scrapping : Roshan Ranawana
scrapping : Pubudu Chaturanga
scrapping : Anarkalli Aakarsha
scrapping : Dilhani Ashokamala
scrapping : Bandu Samarasinghe
scrapping : Yashoda Wimaladharma
scrapping : Jackson Anthony
scrapping : Sangeetha Weeraratne
scrapping : Arjuna Kamalanath
scrapping : Channa Perera
scrapping : Mahendra Perera
scrapping : Tenison Cooray
scrapping : Dulani Anuradha
s

In [117]:
len(artist_list)

88

In [118]:
artist_list = get_artist_list(100,200)

scrapping : Buddhadasa Withanachchi
scrapping : Damith Fonseka
scrapping : Daya Alwis
scrapping : Dharmasiri Bandaranayake
scrapping : H.A. Perera
scrapping : J.H. Jayawardana
scrapping : Janesh Silva
scrapping : Jayalath Manorathne
scrapping : Jayasekara Aponsu
scrapping : Joseph Senevirathna
scrapping : Kumara Thirimadura
scrapping : Lucien Bulathsinhala
scrapping : M.V. Balan
scrapping : Mahesh Jayasinghe
scrapping : Mark Samson
scrapping : Milton Jayawardhana
scrapping : Nawanandana Wijesinghe
scrapping : Pathiraja L.S.Dayananda
scrapping : Palitha Silva
scrapping : Pitipana Silva
scrapping : Quintus Weerakoon
scrapping : Robin Fernando
scrapping : Rodney Warnakula
scrapping : Roger Senewirathna
scrapping : Ronnie Leitch
scrapping : Roshan Pilapitiya
scrapping : Sahan Wijesinghe
scrapping : Sando Haris
scrapping : Sandun Wijesiri
scrapping : Sanjaya Leelarathna
scrapping : Sarath Dikkumbura
scrapping : Sarath Kothalawala
scrapping : Sathischandra Edirisinghe
scrapping : Saumya Liya

In [119]:
len(artist_list)

86

In [120]:
artist_list = get_artist_list(200,250)

scrapping : Basil De Seram
scrapping : Berty Gunathilaka
scrapping : Bimal Jayakody
scrapping : Brandon Grero
scrapping : Buddhi Wickrama
scrapping : Buddhika Jayaratne
scrapping : Buddhika Rambukwella
scrapping : Chandrasiri Kodithuwakku
scrapping : Chanky Ipalawatta
scrapping : Chathura Perera
scrapping : Christy Leonard Perera
scrapping : Chula Mendis
scrapping : Cyril Dharmawardhana
scrapping : D.B. Gangodathanna
scrapping : Damayantha Perera
scrapping : Danny Samaraweera
scrapping : Daya Thennakoon
scrapping : Thakshila Sewwandi
scrapping : Prithi Wijesinghe
scrapping : Ravindra Pradeep
scrapping : Nishani Thennakoon
scrapping : Rohitha Kumara
scrapping : Sameera Madushan
scrapping : Lal Devarathna
scrapping : George Senanayake
scrapping : Shaggy
scrapping : Thanushka Kahandawala
scrapping : Dayasiri Hettiarachchi
scrapping : Deepthi Weerasooriya
scrapping : Dee Ga. Somapala
scrapping : Deepal Gunarathna
scrapping : Deshan Amarasinghe
scrapping : Devinda Marcus
scrapping : Dimuthu

In [121]:
len(artist_list)

36

In [9]:
artist_list = get_artist_list(250,350)

scrapping : Geetal Perera
scrapping : Gihan Fernando
scrapping : Giriraj Kaushalya
scrapping : Gnananga Gunawardana
scrapping : H.S. Perera
scrapping : Harry Wimalasena
scrapping : Harsha Bulathsinghala
scrapping : Harshana Nanayakkara
scrapping : Hemapriya Kandambi
scrapping : Hemantha Eriyagama
scrapping : Indika Fernando
scrapping : Indika Ginige
scrapping : Indrajith Navinna
scrapping : Jagath Benaragama
scrapping : Jagath Chamila
scrapping : Jagath Rohan
scrapping : Jagath Maldeniya
scrapping : Janak Premalal
scrapping : Janaka Kumbukge
scrapping : Janaka Jayakodi
scrapping : Jayalal Rohana
scrapping : Jayantha Ketagoda
scrapping : Jayantha Ranawaka
scrapping : Jayanthadas Perera
scrapping : Jayatissa Athulasiri
scrapping : Jeevan Handunneththi
scrapping : Joe Livera
scrapping : K. Mohankumar
scrapping : K. Mohan Raj
scrapping : K. Senthilkumar
scrapping : K.A. Piyakaru
scrapping : Kalaichelvan
scrapping : Karunarathna Ranasingha
scrapping : Karu Disanayaka
scrapping : Kanchana P.

In [10]:
len(artist_list)

76

In [12]:
artist_list = get_artist_list(350,450)

scrapping : R.P. Ruwan Samantha
scrapping : Rahal Bulathsinghala
scrapping : Rajah Ganesshan
scrapping : Rajasinge Loluwagoda
scrapping : Rajeewa Nanayakkara
scrapping : Rajiendeen
scrapping : Rajitha Hiran Chamikara
scrapping : Ramaiah Sithambaram
scrapping : Ranga Bandaranayaka
scrapping : Rangana Premarathne
scrapping : Ranjith Dayananda
scrapping : Ranjith Gamini Perera
scrapping : Ranjith Peiris
scrapping : Ranjith Rubasinghe
scrapping : Ranjith Jayasekara
scrapping : Ravindra Yasas Kumanayaka
scrapping : Richard Weerakkodi
scrapping : Rinsley Weerarathna
scrapping : Rizvan(FA)
scrapping : Rodney Fraiser
scrapping : Rohana Beddage
scrapping : Rohana Hettiarachchi
scrapping : Rohith Mannege
scrapping : Roshan Ravindra
scrapping : Ruwan Shri Lal Dalpadadu
scrapping : S. Ramadas
scrapping : S. Selvaraja
scrapping : S. Selvesegaran
scrapping : Sahan Ranwala
scrapping : S. Malmalarachchige
scrapping : Saliya Kumara Sri Wasalasinghe
scrapping : Saman Bokalawela
scrapping : Saman Hemarat

In [13]:
len(artist_list)

80

In [14]:
print("Scraping Completed!")

Scraping Completed!
